## Automatizacion de reporte anual
### Jorge Alberto Ramos Garcia



### Importamos librerias

In [59]:
!pip install pandas faker

In [60]:
import pandas as pd
from faker import Faker
from random import randint, choice
import numpy as np
import uuid
faker=Faker("es_MX")

### Funcion para generar tablas

In [61]:
def generar_datos(num_filas):
  """
  Funcion que se encarga de generar datos para crear las tablas Tarjetas, Datos y Compras

  Args:
      num_filas (int): Numero de filas que se quieren generar
  Returns:
      compras_df: DataFrame con los datos de las compras
      Tarjetas_df: DataFrame con los datos de las tarjetas
      datos_df: DataFrame con los datos de los clientes
  """
  Tarjetas=[["Tarjeta","Rango_inferior","Rango_superior"],["Azul",0,4999],["Oro",5000,14999],["Platino",15000,99999],["Black",100000,"nada"]]
  Tarjetas_df=pd.DataFrame(Tarjetas)
  Tarjetas_df.columns=Tarjetas_df.iloc[0]
  Tarjetas_df=Tarjetas_df.drop(0)
  Tarjetas_df = Tarjetas_df.reset_index(drop=True)

  compras=[["Nombre","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiempre","Octubre","Noviempre","Diciembre"]]
  for i in range(num_filas):
    fila=[faker.name()]
    for j in range(len(compras[0])-1):
      fila.append(randint(0,99999))
    compras.append(fila)




  compras_df=pd.DataFrame(compras)
  compras_df.columns=compras_df.iloc[0]
  compras_df=compras_df.drop(0)
  compras_df = compras_df.reset_index(drop=True)

  datos_df=pd.DataFrame()
  datos_df["id"]=[uuid.uuid4() for i in range(len(compras_df["Nombre"]))]
  datos_df["Nombre"]=compras_df["Nombre"]
  datos_df["Tarjeta"]=np.random.choice(Tarjetas_df["Tarjeta"],len(compras_df["Nombre"]))
  datos_df["Direccion"]=[faker.address() for i in range(len(compras_df["Nombre"]))]
  datos_df["Correo"]=[faker.email() for i in range(len(compras_df["Nombre"]))]

  return compras_df,Tarjetas_df,datos_df



### Funciones para generar el reporte

In [62]:
def calculatotal(dataframe):
  """
  Funcion que se encarga de calcular el total de cada cliente
  Args:
      dataframe (DataFrame): DataFrame con los datos de las compras
  Returns:
      df: DataFrame con los datos de las compras y el total
  """
  df=dataframe.copy()
  df["Total"] = df.drop(columns="Nombre").sum(axis=1)
  return df


def genera_resumen(tarjetas,datos,total):
  """
  Funcion que se encarga de generar el resumen de las compras
  Args:
      tarjetas (DataFrame): DataFrame con los datos de las tarjetas
      datos (DataFrame): DataFrame con los datos de los clientes
      total (DataFrame): DataFrame con los datos de las compras y el total
  Returns:
      df: DataFrame con los datos del resumen
  """
  df=pd.DataFrame()
  df["id"]=datos["id"]
  df["Nombre"]=datos["Nombre"]
  df["email"]=datos["Correo"]
  df["Tarjeta"]=datos["Tarjeta"]
  df=df.merge(tarjetas,on="Tarjeta",how="left")
  df["Total"]=total["Total"]
  df["Promedio_mensual"]=df["Total"]/12
  df["Accion"]=np.where(df["Total"]>df["Rango_inferior"],"Aumentar",np.where(df["Total"]<df["Rango_superior"],"Disminuir","Nada"))
  df["estatus_envio"]="PENDIENTE"
  df["timestamp_envio"]=""
  return df



### Funciones que sirven para poder guardar el reporte en google drive

In [63]:
!pip -q install gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [64]:
def guarda_resumen(resumen_df):
  sh = gc.create("BBVA_Demo_Acciones_Fiscal")

  # Crea o toma una hoja llamada "acciones"
  try:
      worksheet = sh.worksheet("acciones")
      worksheet.clear()
  except:
      worksheet = sh.add_worksheet(title="acciones", rows="2000", cols="30")

  # Actualiza la hoja con los datos del DataFrame

  worksheet.update([resumen.columns.tolist()] + resumen.astype(str).values.tolist())
  print("Listo:", sh.url)



## Ejecucion

In [65]:
compras_df,Tarjetas_df,datos=generar_datos(50)

In [66]:
# compras_df=pd.read_excel("ruta_compras")
# Tarjetas_df=pd.read_excel("ruta_tarjetas")
# datos=pd.read_excel("ruta_datos")

In [67]:
Tarjetas_df["Rango_superior"]=pd.to_numeric(Tarjetas_df["Rango_superior"],errors="coerce")
compras_df.iloc[:,1:]=compras_df.iloc[:,1:].apply(pd.to_numeric,errors="coerce")

In [68]:
total=calculatotal(compras_df)
resumen=genera_resumen(Tarjetas_df,datos,total)

In [69]:
guarda_resumen(resumen)

Listo: https://docs.google.com/spreadsheets/d/18Xif_DOVsNHsTgZxiGJkWeUIhjDdFX_SLK17bw7p-_A
